In [ ]:
# A basic mockup of our deep learning neural network model: looking at
# how reviews could determine the sscore of a tv show

In [2]:
# import dependencies
import pandas as pd
from nrclex import NRCLex

In [3]:
#Load CSV.. a text to columns process was already performed to separate the season field. 

file_to_load = 'selected_reviews.csv'

review_df = pd.read_csv(file_to_load)
review_df

,title,season,rank,critic_meta_score,review_content,review_source,author,review_date,summary,meta_score,user_score,release_date,link,critics_reviews_link
0,Rectify,4.0,1,100.0,It allows us to know and care for these charac...,Collider,Allison Keene,27-Oct-16,Daniel tries to start a new life outside of Pa...,99,8.7,26-Oct-16,https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
1,Rectify,4.0,1,100.0,"Rectify, a drama entering its final season on ...",The New York Times,James Poniewozik,25-Oct-16,Daniel tries to start a new life outside of Pa...,99,8.7,26-Oct-16,https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
2,Rectify,4.0,1,100.0,No other series so poignantly probes the human...,Salon,Melanie McFarland,26-Oct-16,Daniel tries to start a new life outside of Pa...,99,8.7,26-Oct-16,https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
3,Rectify,4.0,1,100.0,None of these characters is particularly happy...,Yahoo TV,Ken Tucker,26-Oct-16,Daniel tries to start a new life outside of Pa...,99,8.7,26-Oct-16,https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
4,Rectify,4.0,1,100.0,Rectify is the best series I have ever seen on...,The Daily Beast,Malcolm Jones,26-Oct-16,Daniel tries to start a new life outside of Pa...,99,8.7,26-Oct-16,https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52889,The 1/2 Hour News Hour,1.0,3139,0.0,You couldn't skewer a cube of tofu with materi...,Slate,Troy Patterson,NaN,"The right-wing answer to ""The Daily Show With ...",13,2.8,18-Feb-07,https://www.metacritic.com/tv/the-12-hour-news...,https://www.metacritic.com/tv/the-12-hour-news...
52890,The 1/2 Hour News Hour,1.0,3139,0.0,"It's hard not to keep staring perplexed, squin...",The New Republic,Sacha Zimmerman,NaN,"The right-wing answer to ""The Daily Show With ...",13,2.8,18-Feb-07,https://www.metacritic.com/tv/the-12-hour-news...,https://www.metacritic.com/tv/the-12-hour-news...
52891,The 1/2 Hour News Hour,1.0,3139,0.0,It's even worse than anyone imagined.,The A.V. Club,Amelie Gillette,NaN,"The right-wing answer to ""The Daily Show With ...",13,2.8,18-Feb-07,https://www.metacritic.com/tv/the-12-hour-news...,https://www.metacritic.com/tv/the-12-hour-news...
52892,The 1/2 Hour News Hour,1.0,3139,10.0,The humor is so predictable and so stale that ...,Chicago Tribune,Maureen Ryan,NaN,"The right-wing answer to ""The Daily Show With ...",13,2.8,18-Feb-07,https://www.metacritic.com/tv/the-12-hour-news...,https://www.metacritic.com/tv/the-12-hour-news...


In [10]:
# Check how many unique shows are in the dataframe 
print(f'There are {len(review_df.title.unique())} different shows')

There are 2626 different shows


In [22]:
#find shows with multiple seasons

multiple_seasons = df.groupby('title').nunique()
multiple_seasons = multiple_seasons[multiple_seasons['season']>2]['season']

multiple_seasons.sort_values(ascending = False).head(10)

len(multiple_seasons)


112

In [23]:
#fill null values in season column with 0
#These programs are tv_specials and mini series. Perhaps the should be dropped.

df['season'].fillna(0, inplace = True)
df['title'][df['season']==0].unique()

 

array(['Bo Burnham: Inside', 'Planet Earth: Blue Planet II',
       'Homecoming: A Film by Beyoncé', 'The Underground Railroad',
       'Romeo & Juliet', "It's a Sin", 'A Parks and Recreation Special',
       'Longford', 'Kurt Cobain: Montage of Heck', 'Muhammad Ali',
       'Springsteen on Broadway', 'Deadwood: The Movie',
       'Elizabeth Is Missing', 'The Trip to Bountiful', 'Prohibition',
       'Sing Your Song',
       'The Black Church: This Is Our Story, This Is Our Song',
       'Hemingway', 'Between the World and Me',
       'Bright Lights: Starring Carrie Fisher and Debbie Reynolds',
       'The Normal Heart', 'Six by Sondheim',
       "What's My Name: Muhammad Ali", 'The Crime of the Century',
       'The Good Lord Bird', 'Elvis Presley: The Searcher',
       'Immigration Nation', 'The Dresser (2016)', 'Leaving Neverland',
       'Euphoria Special Episode Part 1: Rue',
       'No Direction Home: Bob Dylan', 'Exterminate All the Brutes',
       'Whoopi Goldberg Presents Moms

In [11]:
# Check how many null values are in the dataframe
review_df.isnull().sum()

title                      0
season                  3251
rank                       0
critic_meta_score        205
review_content            27
review_source            205
author                   205
review_date             9956
summary                    0
meta_score                 0
user_score                 0
release_date               0
link                       0
critics_reviews_link       0
dtype: int64

In [12]:
# add the non null values and add it back to the dataframe 
review_df=review_df[review_df.review_content.notnull()]

review_df.isnull().sum()

title                      0
season                  3251
rank                       0
critic_meta_score        178
review_content             0
review_source            178
author                   178
review_date             9929
summary                    0
meta_score                 0
user_score                 0
release_date               0
link                       0
critics_reviews_link       0
dtype: int64

In [27]:
reviews = df['review_content']

reviews = reviews.dropna()

reviews.isnull().sum()

df['reviews_new'] = reviews

In [14]:
sample_size = 1000
sample_df = review_df.head(sample_size)
reviews =review_df['review_content'][0:sample_size].tolist()
print(len(reviews))
print(len(sample_df))

1000
1000


In [15]:
emotions_scores = []

In [16]:
#loop to genearate a dataframe of emotion scores of reviews

for review in reviews:
    text_object = NRCLex(review)
    data = text_object.raw_emotion_scores
    af = text_object.affect_frequencies
    
    emotions_scores.append(af)

    
print(emotions_scores)

[{'fear': 0.08333333333333333, 'anger': 0.0, 'anticip': 0.0, 'trust': 0.08333333333333333, 'surprise': 0.08333333333333333, 'positive': 0.25, 'negative': 0.08333333333333333, 'sadness': 0.16666666666666666, 'disgust': 0.0, 'joy': 0.08333333333333333, 'anticipation': 0.16666666666666666}, {'fear': 0.2857142857142857, 'anger': 0.14285714285714285, 'anticip': 0.0, 'trust': 0.0, 'surprise': 0.0, 'positive': 0.14285714285714285, 'negative': 0.14285714285714285, 'sadness': 0.2857142857142857, 'disgust': 0.0, 'joy': 0.0}, {'fear': 0.041666666666666664, 'anger': 0.0, 'anticip': 0.0, 'trust': 0.20833333333333334, 'surprise': 0.041666666666666664, 'positive': 0.20833333333333334, 'negative': 0.041666666666666664, 'sadness': 0.08333333333333333, 'disgust': 0.041666666666666664, 'joy': 0.16666666666666666, 'anticipation': 0.16666666666666666}, {'fear': 0.0, 'anger': 0.06666666666666667, 'anticip': 0.0, 'trust': 0.2, 'surprise': 0.0, 'positive': 0.13333333333333333, 'negative': 0.2, 'sadness': 0.06

In [17]:
# convert emotion list into a dataframe
emotions_df = pd.DataFrame(emotions_scores)
emotions_df

,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,0.083333,0.000000,0.0,0.083333,0.083333,0.250000,0.083333,0.166667,0.000000,0.083333,0.166667
1,0.285714,0.142857,0.0,0.000000,0.000000,0.142857,0.142857,0.285714,0.000000,0.000000,NaN
2,0.041667,0.000000,0.0,0.208333,0.041667,0.208333,0.041667,0.083333,0.041667,0.166667,0.166667
3,0.000000,0.066667,0.0,0.200000,0.000000,0.133333,0.200000,0.066667,0.133333,0.133333,0.066667
4,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.250000,0.250000
996,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
997,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,NaN
998,0.000000,0.000000,0.0,0.500000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,NaN


In [18]:
# merge the sampled datframe to the emotions dataframe
df_merged = sample_df.merge( emotions_df, left_index = True, right_index = True)

df_merged

,title,season,rank,critic_meta_score,review_content,review_source,author,review_date,summary,meta_score,...,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,Rectify,4.0,1,100.0,It allows us to know and care for these charac...,Collider,Allison Keene,27-Oct-16,Daniel tries to start a new life outside of Pa...,99,...,0.000000,0.0,0.083333,0.083333,0.250000,0.083333,0.166667,0.000000,0.083333,0.166667
1,Rectify,4.0,1,100.0,"Rectify, a drama entering its final season on ...",The New York Times,James Poniewozik,25-Oct-16,Daniel tries to start a new life outside of Pa...,99,...,0.142857,0.0,0.000000,0.000000,0.142857,0.142857,0.285714,0.000000,0.000000,NaN
2,Rectify,4.0,1,100.0,No other series so poignantly probes the human...,Salon,Melanie McFarland,26-Oct-16,Daniel tries to start a new life outside of Pa...,99,...,0.000000,0.0,0.208333,0.041667,0.208333,0.041667,0.083333,0.041667,0.166667,0.166667
3,Rectify,4.0,1,100.0,None of these characters is particularly happy...,Yahoo TV,Ken Tucker,26-Oct-16,Daniel tries to start a new life outside of Pa...,99,...,0.066667,0.0,0.200000,0.000000,0.133333,0.200000,0.066667,0.133333,0.133333,0.066667
4,Rectify,4.0,1,100.0,Rectify is the best series I have ever seen on...,The Daily Beast,Malcolm Jones,26-Oct-16,Daniel tries to start a new life outside of Pa...,99,...,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,The Americans,3.0,59,100.0,The Americans not only built on its impressive...,The Hollywood Reporter,Tim Goodman,29-Jan-15,The third season picks up late in 1982 with El...,92,...,0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.250000,0.250000
996,The Americans,3.0,59,100.0,"The Americans is only getting better with age,...",IndieWire,Ben Travers,26-Jan-15,The third season picks up late in 1982 with El...,92,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
997,The Americans,3.0,59,100.0,The first four episodes of Season 3 of The Ame...,Washington Post,Hank Stuever,27-Jan-15,The third season picks up late in 1982 with El...,92,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,NaN
998,The Americans,3.0,59,100.0,The first four episodes of Season 3 are every ...,The Huffington Post,Maureen Ryan,28-Jan-15,The third season picks up late in 1982 with El...,92,...,0.000000,0.0,0.500000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,NaN


In [54]:
df_merged.to_csv('tv_emotions_data.csv')